In [35]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import time
import warnings ; warnings.warn = lambda *args,**kwargs: None

In [41]:


example_events = [
    {
        'date time': '04/20/24 01:18:22',
        'event':'START RECORD', 
        'classification':'start record'
    },
    {
        'date time':'04/20/24 00:30:00',
        'event':'go to sleep'
    }
]

def timestamp():
    return datetime.now().strftime("[%m/%d/%y - %H:%M:%S] ")
def get_date():
    return datetime.now().strftime("%m/%d/%y")
def get_time():
    return datetime.now().strftime("%H:%M:%S")
def get_date_time():
    return datetime.now().strftime("%m/%d/%y %H:%M:%S")
def get_time_difference(entry1,entry2,verbose=False,unit='minutes'):
    t1_string = entry1['date time']
    t1 = datetime.strptime(t1_string, "%m/%d/%y %H:%M:%S")

    t2_string = entry2['date time']
    t2 = datetime.strptime(t2_string, "%m/%d/%y %H:%M:%S")
    if unit == 'minutes':
        factor=60
    elif unit == 'seconds':
        factor=1
    difference_minutes = int((t2-t1).seconds/factor)
    if verbose:
        print(f"start: ".rjust(12)+f"{t1}")
        print(f"finish: ".rjust(12)+f"{t2}")
        print(f"difference: ".rjust(12)+f"{difference_minutes:.0f} {unit}")
        
    return difference_minutes

class events_analyzer():
    def __init__(self, records = None, verbose = False):
        self.verbose = verbose
        self.print(f"\n{timestamp()} INITIALIZING EVENTS ANALYZER")
        self.events = pd.DataFrame(columns=['date time','order','event','classification','quantity'])

        if not pd.isnull(records).any():
            for entry in records:
                self.append_entry(entry)

        else:
            entry = {
                'event':'START RECORD',
                'classification':'start record'
            }
            self.append_entry(entry)
    def print(self,text,end='\n'):
        if self.verbose:
            print(text,end=end)
    def append_entry(self,entry):
        if 'date time' not in entry.keys():
            entry['date time'] = get_date_time()
        self.print(f"\n{timestamp()} APPEND NEW ENTRY:")
        self.print(json.dumps(entry,indent=4))
        self.events.loc[self.events.index.size] = entry
        self.order()

    def wake_up(self, date_time = get_date_time()):
        entry = {}
        entry['date time'] = date_time
        self.print(f"\n{timestamp()} RECORDING WAKE UP",end="")
        entry['event'] = 'wake up'
        entry['classification'] = 'end sleep'

        starts = ['START RECORD','go to sleep']
        nstarts = len(set(starts).intersection(set(self.events['event'])))

        if nstarts>0:
            start_entry = dict(self.events[self.events['event'].isin(starts)].iloc[-1])
            difference = get_time_difference(start_entry,entry)
            self.print(f" (Sleep time = {difference} minutes)")
            get_time_difference(start_entry,entry,verbose=self.verbose)
            entry['quantity'] = f"{difference} minutes"
        else:
            self.print("")
        self.append_entry(entry)
        
    def go_to_sleep(self, date_time = get_date_time()):
        entry = {}
        entry['date time'] = date_time
        self.print(f"\n{timestamp()} RECORDING GO TO SLEEP")
        entry['event'] = 'go to sleep'
        entry['classification'] = 'start sleep'
        self.append_entry(entry)
        
    def display(self):
        self.print(f"\n{timestamp()} DISPLAY EVENT RECORD:\n")
        print(self.events)
        
    def order(self):
        unordered_index = self.events[self.events['order'].isnull()].index
        unordered_timestamps = self.events['date time'][unordered_index].unique()
        for timestamp in unordered_timestamps:
            mask = self.events['date time']==timestamp
            self.events['order'][mask] = range(1,sum(mask)+1)
        self.events['order'] = self.events['order'].astype(int)
        
test = events_analyzer(records=example_events,verbose=True)
test.wake_up()
test.display()


[04/20/24 - 01:27:05]  INITIALIZING EVENTS ANALYZER

[04/20/24 - 01:27:05]  APPEND NEW ENTRY:
{
    "date time": "04/20/24 01:18:22",
    "event": "START RECORD",
    "classification": "start record"
}

[04/20/24 - 01:27:05]  APPEND NEW ENTRY:
{
    "date time": "04/20/24 00:30:00",
    "event": "go to sleep"
}

[04/20/24 - 01:27:05]  RECORDING WAKE UP (Sleep time = 57 minutes)
     start: 2024-04-20 00:30:00
    finish: 2024-04-20 01:27:05
difference: 57 minutes

[04/20/24 - 01:27:05]  APPEND NEW ENTRY:
{
    "date time": "04/20/24 01:27:05",
    "event": "wake up",
    "classification": "end sleep",
    "quantity": "57 minutes"
}

[04/20/24 - 01:27:05]  DISPLAY EVENT RECORD:

           date time  order         event classification    quantity
0  04/20/24 01:18:22      1  START RECORD   start record         NaN
1  04/20/24 00:30:00      1   go to sleep            NaN         NaN
2  04/20/24 01:27:05      1       wake up      end sleep  57 minutes


In [37]:
example_entry1 = {
    'date time':'04/20/24 00:30:00',
    'event':'wake up'
}

example_entry2 = {
    'date time':'04/20/24 00:45:00',
    'event':'wake up'
}

get_time_difference(example_entry1,example_entry2,verbose=True,unit='seconds')

     start: 2024-04-20 00:30:00
    finish: 2024-04-20 00:45:00
difference: 900 seconds


900